In [3]:
import os
import requests
import pandas as pd


In [4]:
#Éstas son las direcciones que descargan los archivos en formato csv.

#Temperatura mínima en el 2019
url1 ='https://smn.conagua.gob.mx/tools/RESOURCES/com_archivo_datos_resumenes/201901010000TMin.csv'

#Temperatura máxima en el 2019
url2 ='https://smn.conagua.gob.mx/tools/RESOURCES/com_archivo_datos_resumenes/201901010000TMax.csv'

In [5]:
tminima = pd.read_csv(url1, encoding='Latin')

tmaxima = pd.read_csv(url2, encoding='Latin')

In [6]:
#podemos observar los primeros datos 
tminima.head()

,Lon,Lat,Clave,Edo,Est,Tmin
0,-99.75,16.76,76805,GRO,ACAPULCO,21.606452
1,-111.83,30.71,76113,SON,ALTAR,6.055172
2,-93.90,16.24,76840,CHIS,ARRIAGA,21.712903
3,-90.50,19.83,76695,CAMP,CAMPECHE,17.348148
4,-111.65,25.01,76402,BCS,CD. CONSTITUCIÓN,8.545161


In [7]:
tmaxima.head()

,Lon,Lat,Clave,Edo,Est,Tmax
0,-99.75,16.76,76805,GRO,ACAPULCO,33.277419
1,-102.29,21.85,76571,AGS,AGUASCALIENTES,22.941935
2,-111.83,30.71,76113,SON,ALTAR,20.868966
3,-93.90,16.24,76840,CHIS,ARRIAGA,34.464516
4,-90.50,19.83,76695,CAMP,CAMPECHE,29.400000


In [8]:
#O seleccionar el número de datos que quermos ver (de arriba hacia abajo), por ejemplo 10
tminima.head(10)

,Lon,Lat,Clave,Edo,Est,Tmin
0,-99.75,16.76,76805,GRO,ACAPULCO,21.606452
1,-111.83,30.71,76113,SON,ALTAR,6.055172
2,-93.90,16.24,76840,CHIS,ARRIAGA,21.712903
3,-90.50,19.83,76695,CAMP,CAMPECHE,17.348148
4,-111.65,25.01,76402,BCS,CD. CONSTITUCIÓN,8.545161
5,-109.92,27.48,76258,SON,CD. OBREGON,7.353571
6,-99.17,23.75,76491,TAMPS,CD. VICTORIA,10.256000
7,-99.20,19.40,76680,DF,CENTRAL TACUBAYA,8.629032
8,-88.30,18.48,76750,QROO,CHETUMAL,18.380645
9,-106.13,28.67,76225,CHIH,CHIHUAHUA,3.293548


In [80]:
# Para arreglar los índices
tmaxima . set_index ([  'Edo' ])

,Lon,Lat,Clave,Est,Tmax
Edo,,,,,
GRO,-99.75,16.76,76805,ACAPULCO,33.277419
AGS,-102.29,21.85,76571,AGUASCALIENTES,22.941935
SON,-111.83,30.71,76113,ALTAR,20.868966
CHIS,-93.90,16.24,76840,ARRIAGA,34.464516
CAMP,-90.50,19.83,76695,CAMPECHE,29.400000
...,...,...,...,...,...
MEX,-99.80,19.33,TBRMX,Tres Barrancas Méx.,19.673077
MEX,-98.90,19.44,TJCMX,El Tejocote Méx.,21.461538
MEX,-99.74,19.59,SLBMX,San Bartolo del Llano Méx.,19.865385


In [81]:
#Si quiero veer partes específicas de la tabla puedo crear un atabla pivote
tmaxima.pivot_table('Tmax', index='Edo')

,Tmax
Edo,
AGS,23.305760
BC,20.565238
BCS,25.219430
CAMP,29.285450
CHIH,20.017167
CHIS,29.346937
COAH,20.348884
COL,32.650036
DF,22.683031


In [26]:
#Puedo guardar esas partes específicas como una nueva tabla para trabajar con ellas
t_max = tmaxima.pivot_table('Tmax', index='Edo')
t_min = tminima.pivot_table('Tmin', index='Edo')

In [47]:
#Hago una fusión de las tablas guardadas para tener la temperatura máxima y mínima de cada estado en una sola tabla
merged = pd.merge(t_max, t_min, how='outer',
                  left_on='Edo', right_on='Edo')
#merged = merged.drop('Edo', 1) # drop duplicate info
merged.head()

,Tmax,Tmin
Edo,,
AGS,23.305760,4.566749
BC,20.565238,8.650000
BCS,25.219430,13.905677
CAMP,29.285450,17.311654
CHIH,20.017167,0.980009


In [54]:
#Para buscar si hay estados sin datos verificamos
merged.isnull().any()

Tmax     True
Tmin    False
dtype: bool

In [55]:
#vemos que hay datos faltantes en Tmax y buscamos cuáles son
merged[merged['Tmax'].isnull()].head(5)

,Tmax,Tmin
Edo,,
QRO,NaN,9.322581


In [56]:
#Así podemos ignorar los datos faltantes
merged. dropna ( inplace = True ) 

In [59]:
#Podemos hacer operaciones entre las columnas de la tabla fusionada, por ejemplo la diferencia de temperatura máxima y mínima de cada estado
#merged.set_index('Edo', inplace=True)
diferencia = merged['Tmax'] - merged['Tmin']

In [60]:
diferencia.sort_values(ascending=False, inplace=True)
diferencia.head()
#Con esto nos podemos dar cuanta que el estado con más diferenica de temperatura fue durango

Edo
DGO     19.618807
CHIH    19.037158
MICH    19.029432
TLAX    18.877688
AGS     18.739011
dtype: float64

In [65]:
#Tmbién podemos ver los estados con menor diferencia yendo al final de la tabla
diferencia.tail()

Edo
QROO    11.700403
BCS     11.313754
SLP     11.256903
VER     10.249251
TAB      9.398468
dtype: float64

In [70]:
#La temperatura mínima, máxima y la diferencia promedio:
t_min.mean()

Tmin    10.126373
dtype: float64

In [71]:
t_max.mean()

Tmax    25.295201
dtype: float64

In [72]:
diferencia.mean()

15.142899828320106

In [73]:
#De la tabla fusionada calculamos el promedio resultante de la temperatura máxima y mínima
merged.mean(axis='columns')

Edo
AGS      13.936255
BC       14.607619
BCS      19.562553
CAMP     23.298552
CHIH     10.498588
CHIS     22.369124
COAH     13.231084
COL      25.819440
DF       15.411652
DGO      12.695142
GRO      22.748404
GTO      15.508780
HGO      14.993390
JAL      18.029378
MEX      12.816365
MICH     17.205167
MOR      17.689391
NAY      22.356855
NL       14.213307
OAX      21.396129
PUE      15.114949
QROO     23.977928
SIN      20.556872
SLP      19.477241
SON      14.376226
TAB      23.350393
TAMPS    17.513564
TLAX     12.394758
VER      18.953760
YUC      22.524498
ZAC      12.808925
dtype: float64

In [76]:
#Para saber el número total de elementos, por ejemplo para T_min
t_min.count()
#Sería el total de estados analizados.

Tmin    32
dtype: int64

In [79]:
#La varianza en T_min
t_min.var()

Tmin    26.725197
dtype: float64